In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 574 kB/s 
     |████████████████████████████████| 5.8 MB 60.5 MB/s 
     |████████████████████████████████| 1.3 MB 61.1 MB/s 
     |████████████████████████████████| 182 kB 78.7 MB/s 
     |████████████████████████████████| 7.6 MB 49.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c0e2c3697a8fcb23729b51eb6374dd38bdf1a12fc30126409230a678017a4598
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
import pandas as pd
import numpy as np
import pickle

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!ls

 1202BERT.ipynb		  emo_dict.pkl		     RNN_감정분류.ipynb
 1207jw.ipynb		  영화코사인유사도.ipynb     simple_chatbot.ipynb
 1208SentenceBERT.ipynb   movies_embedded.csv	     tmdb_5000_movies.csv
 chatbot_data2.csv	  movies.ipynb
'Colab Notebooks'	  naver_movie_review.ipynb


In [ ]:
with open('emo_dict.pkl','rb') as f:
  emo_dict = pickle.load(f)

In [ ]:
chatbot_embedded = pd.read_csv('chatbot_data2.csv')
movies_embedded = pd.read_csv('movies_embedded.csv')

In [ ]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/585 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
chatbot_embedded.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
chatbot_embedded['embedding'] = chatbot_embedded.apply(lambda x: model.encode(x['사람']), axis=1)

In [ ]:
movies_embedded = pd.read_csv('movies_embedded.csv')
movies_embedded['embedding'] = movies_embedded.apply(lambda x: model.encode(x['제목']), axis=1)

In [ ]:
while(1):
  text = str(input("user> "))
  if text == '이제 그만해.':
    print('bot> 넵..')
    break
  embedded = model.encode(text)

  chatbot_embedded['score'] = chatbot_embedded.apply(lambda x: cos_sim(x['embedding'], embedded), axis=1)
  genre = chatbot_embedded.iloc[chatbot_embedded['score'].idxmax(), 1]
  emotion = emo_dict[genre]
  print(f'bot> {emotion} 때 보기 좋은 영화 추천해 드릴게요.')

  movies_embedded1 = movies_embedded.copy()
  movies_embedded1 = movies_embedded1[movies_embedded1['영화'] == genre]
  movies_embedded1['score'] = movies_embedded1.apply(lambda x: cos_sim(x['embedding'], embedded), axis=1)

  top3 = movies_embedded1.sort_values('score', ascending=False).iloc[:3, 0]

  for i, t in enumerate(top3.values):
    print(f'{i+1}. {t}')
  print('='*20)

user> 오늘 친구랑 맛있는 음식을 먹었어
bot> 기쁠 때 보기 좋은 영화 추천해 드릴게요.
1. 맛있는 녀석들
2. 우아한 만찬
3. 고 피쉬!
user> 오늘 버스를 놓쳐서 지각을 했어
bot> 불안할 때 보기 좋은 영화 추천해 드릴게요.
1. 지옥행 특급택시
2. 브레이크
3. 에브리타임 아이 다이
user> 이제 그만해.
bot> 넵..
